# Fabryka abstrakcyjna

**Fabryka abstrakcyjna** (ang. abstract factory) to wzorzec projektowy zapewniający mechanizm tworzenia instancji różnych klas należących do pewnej rodziny bez określania ich konkretnej klasy. Umożliwia on definiowanie interfejsu dla tworzenia obiektów z danej grupy, a właściwe implementacje dostarczają instancje odpowiednich klas. Fabryka abstrakcyjna dzięki temu wspiera zasadę odwrócenia zależności i ułatwia zarządzanie złożonymi systemami poprzez oddzielenie logiki tworzenia obiektów od ich użycia. Fabryka abstrakcyjna jest często stosowana w sytuacjach, gdy istnieje potrzeba dynamicznej zmiany zestawu tworzonych obiektów.

Przeznaczenie i zastosowanie:
- oddzielenie logiki tworzenia obiektów od ich użycia: zmniejsza liczebność zależności w kodzie,
- tworzenie grup powiązanych obiektów: zapewnia spójność w wykorzystaniu zestawów klas,
- modularność kodu: prosta wymiana rodzin obiektów za pomocą zmiany implementacji fabryki.

Założenia początkowe:
- istnieje rodzina fabryk, które zwracają produkt,
- każda fabryka jest realizowana za pomocą innej klasy o tym samym kontrakcie,
- każda fabryka potrafi tworzyć te same rodzaje obiektów,
- każda fabryka zawiera metody o tej samej nazwie,
- celem fabryki abstrakcyjnej będzie dostarczenie odpowiedniej fabryki za pomocą instancji lub referencji.

<img src="img/Abstract_Factory_Design_Pattern_UML.jpg">

<img src="img/Abstract_factory_UML_class.svg">

## Implementacja

Cel: utworzenie fabryki abstrakcyjnej dostarczającej instancji fabryki produkującej dane auto o wskazanej specyfikacji.

In [ ]:
from abc import ABC, abstractmethod
from dataclasses import dataclass, field, asdict
from typing import Any

Klasy składowych części samochodów

In [ ]:
@dataclass
class Wheel:
    diameter: int
    material: str = field(default="aluminium")

In [ ]:
@dataclass
class Body:
    color: str
    thickness: float = field(default=.6)

In [ ]:
@dataclass
class Door:
    interior_material: str
    control: str = field(default="manual")

In [ ]:
@dataclass
class Seat:
    material: str
    control: str = field(default="manual")

In [ ]:
@dataclass
class PremiumSticker:
    icon: str
    color: str

Klasy fabryk odpowiedzialnych za produkcję części składowych. Dobrze jest trzymać wszystkie klasy fabryk w jednym pakiecie tak, aby nie trzeba było za każdym razem powielać ich importów w różnych miejsach w kodzie.

In [ ]:
class Factory(ABC):
    @abstractmethod
    def produce_wheels(self, diameter: int, amount: int) -> tuple:
        pass

    @abstractmethod
    def produce_body(self, color: str) -> Body:
        pass

    @abstractmethod
    def produce_doors(self, interior_material: str, amount: int) -> tuple:
        pass

    @abstractmethod
    def produce_seats(self, material: str, amount: int) -> tuple:
        pass

In [ ]:
class PoloCarFactory(Factory):
    def produce_wheels(self,  diameter: int, amount: int) -> tuple:
        return tuple([Wheel(diameter=diameter) for _ in range(amount)])
    
    def produce_body(self, color: str) -> Body:
        return Body(color=color)
    
    def produce_doors(self, interior_material: str, amount: int) -> tuple:
        return tuple([Door(interior_material=interior_material) for _ in range(amount)])
    
    def produce_seats(self, material: str, amount: int) -> tuple:
        return tuple([Seat(material=material) for _ in range(amount)])

In [ ]:
class GolfCarFactory(Factory):
    def produce_wheels(self,  diameter: int, amount: int) -> tuple:
        return tuple([Wheel(diameter=diameter) for _ in range(amount)])
    
    def produce_body(self, color: str) -> Body:
        return Body(color=color, thickness=2.)
    
    def produce_doors(self, interior_material: str, amount: int) -> tuple:
        return tuple([Door(interior_material=interior_material, control="electric") for _ in range(amount)])
    
    def produce_seats(self, material: str, amount: int) -> tuple:
        return tuple([Seat(material=material) for _ in range(amount)])

In [ ]:
class PasseratiCarFactory(Factory):
    def produce_wheels(self,  diameter: int, amount: int) -> tuple:
        return tuple([Wheel(diameter=diameter, material="steel") for _ in range(amount)])
    
    def produce_body(self, color: str) -> Body:
        return Body(color=color, thickness=2.5)
    
    def produce_doors(self, interior_material: str, amount: int) -> tuple:
        return tuple([Door(interior_material=interior_material, control="electric") for _ in range(amount)])
    
    def produce_seats(self, material: str, amount: int) -> tuple:
        return tuple([Seat(material=material, control="electric") for _ in range(amount)])

Klasa **fabryki abstrakcyjnej**. Klasa ta zwraca instancję fabryki produkującej poszczególne części składowe auta.

In [ ]:
class AbstractFactory:
    @staticmethod
    def get_factory(model: Any) -> Any:
        match model:
            case "Polo":
                return PoloCarFactory()
            case "Golf":
                return GolfCarFactory()
            case "Passat":
                return PasseratiCarFactory()
            case _:
                raise ValueError("Incorrect car model")

Klasy reprezentujące warianty samochodów. Każdy z modeli aut może być dostępny w wersji Basic oraz Premium, które różnią się wyposażeniem dodatkowym.

In [ ]:
@dataclass
class Car:
    wheels: tuple
    body: Body
    doors: tuple
    seats: tuple

In [ ]:
@dataclass
class BasicCar(Car):
    pass

In [ ]:
@dataclass
class PremiumCar(BasicCar):
    sticker: PremiumSticker

Klasa producenta aut.

In [ ]:
class CarManufacturer(ABC):
    client_options: dict

    def __init__(self, client_options: dict) -> None:
        self.client_options = client_options
    
    def produce_car(self) -> BasicCar:
        factory = AbstractFactory.get_factory(self.client_options["model"])
        wheels, body, doors, seats = self._request_parts(factory)

        return BasicCar(wheels=wheels, body=body, doors=doors, seats=seats)

    def _request_parts(self, factory: Any) -> tuple:
        wheels = factory.produce_wheels(self.client_options["diameter"], 4)
        body = factory.produce_body(self.client_options["color"])
        doors = factory.produce_doors(self.client_options["doors"], 5)
        seats = factory.produce_seats(self.client_options["seats"], 5)

        return wheels, body, doors, seats

In [ ]:
class CarBrandBasic(CarManufacturer):
    pass

In [ ]:
class CarBrandPremium(CarManufacturer):
    def produce_car(self) -> PremiumCar:
        basic_car = super().produce_car()
        premium_sticker = self._produce_premium_sticker()
        
        return PremiumCar(**asdict(basic_car), sticker=premium_sticker)

    def _produce_premium_sticker(self) -> PremiumSticker:
        return PremiumSticker(**self.client_options["sticker"])

Klasa klienta odpowiedzialna za złożenie zamówienia na auto na podstawie przekazanej specyfikacji.

In [ ]:
class Client:
    @staticmethod
    def request_car(request: dict) -> Car:
        manufacturer = CarBrandPremium(request)
        new_car = manufacturer.produce_car()

        return new_car

In [ ]:
car_specification = {
    "model": "Golf",
    "diameter": 18,
    "color": "black",
    "doors": "plastic",
    "seats": "normal",
    "sticker": {
        "icon": "apple",
        "color": "gray",
    }
}

In [ ]:
client = Client()
client.request_car(car_specification)

## Podsumowanie

Fabryka abstrakcyjna jest wykorzystywana w przypadkach, gdy istnieje potrzeba utworzenia instancji różnych klas należących do pewnej rodziny bez określania ich konkretnej klasy. Takie podejście rodzi pewne konsekwencje, które można podzielić na pozytywne i negatywne.

Wady:
- trudnośi w dodawaniu nowych klas do fabryki,
- wszystkie metody wykonywane są na tej samej klasie/obiekcie.

Zalety:
- hermetyzacja procesu tworzenia obiektów danego typu,
- logika procesu tworzenia obiektów i korzystania z nich jest oddzielona i ogólnodostępna,
- zmniejszona objętość kodu.
